In [1]:
import itertools
import lime_explainer as lime

/usr/local/envs/py3env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
feature_names = ['customer_id', 'monetary_dnn', 'monetary_btyd', 'frequency_dnn',
                 'frequency_btyd', 'recency', 'T', 'time_between',
                 'avg_basket_value', 'avg_basket_size', 'cnt_returns',
                 'has_returned', 'frequency_btyd_clipped', 'monetary_btyd_clipped',
                 'target_monetary_clipped', 'target_monetary']

In [3]:
explainer = lime.Explainer()

In [5]:
explainer.load_data(
  gcs_path = 'gs://ml-clv-data/model_data.csv',
  features_to_use = [1,3,5,6,7,8,9,10,11],
  feature_names = feature_names,
  target_idx = 15,
  skip_first=True 
)

In [6]:
explainer.create_cmle_client(
  gcp_project = 'ml-clv',
  gcp_model = 'dnn_airflow',
  gcp_model_version = 'v1',
  output_func = lambda x : x['predicted_monetary'] 
)

In [7]:
record, df = explainer.explain_random_record(numeric_rows=[1,2,3,7,8])
print(record)
df

Intercept 7943.551373809409
Prediction_local [6636.29646203]
Right: 5343.44189453125
[4025.64, 44.0, 256.0, 256.0, 5.82, 91.49, 40.73, 0.0, 0.0]


,representation,weight
0,has_returned <= 0.00,-7224.636475
1,recency <= 291.00,3778.620757
2,cnt_returns <= 0.00,2153.161151
3,29.00 < frequency_dnn <= 52.00,-377.203222
4,2423.27 < monetary_dnn <= 4087.26,362.802878


In [8]:
mod_record = ','.join(['2000'] + [str(e) for e in record[1:]])
print('Mod Record: {}\n\n'.format(mod_record))
_, mod_df = explainer.explain_record(mod_record)
mod_df

Mod Record: 2000,44.0,256.0,256.0,5.82,91.49,40.73,0.0,0.0


Intercept 5729.9626205278055
Prediction_local [2220.78516684]
Right: 1979.90869140625


,representation,weight
0,cnt_returns <= 0.00,-2028.910105
1,has_returned <= 0.00,-1276.689719
2,29.00 < frequency_dnn <= 52.00,739.794669
3,avg_basket_value > 66.77,-738.071122
4,T <= 306.00,-205.301176
